In [1]:
import joblib
from sklearn.model_selection import train_test_split

import configuration.configuration as configuration
from mel_spectrogram.mel_spectrogram import MelSpectrogramGenerator
from notebook_utils import notebookToPython

In [2]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file
Generating new model name: ASVspoof-2019-1_2025-03-16T21-46-18.789612.libjob
Assigned model name: ASVspoof-2019-1_2025-03-16T21-46-18.789612.libjob


In [3]:
generator = MelSpectrogramGenerator()
X, y_encoded = generator.generateMelSpectrograms(job, job.dataPathSuffix)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...
fullDataPath: C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac
Loading audio files: 1269
Loading audio files: 2538
Loading audio files: 3807
Loading audio files: 5076
Loading audio files: 6345
Loading audio files: 7614
Loading audio files: 8883
Loading audio files: 10152
Loading audio files: 11421
Loading audio files: 12690
Loading audio files: 13959
Loading audio files: 15228
Loading audio files: 16497
Loading audio files: 17766
Loading audio files: 19035
Loading audio files: 20304
Loading audio files: 21573
Loading audio files: 22842
Loading audio files: 24111
Loading audio files: 25380
Number of audio files load: 25380


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)    # test data is 20% of all data

In [5]:
import model_definitions.model_cnn_definition as model_cnn_definition

modelDef = model_cnn_definition.ModelCnnDefinition(job, X_train.shape[2], 1)
model = modelDef.buildModel()

__job__: <configuration.configuration.Job object at 0x000002307D3DE570>


In [6]:
model.compile(optimizer=job.optimizer, loss=job.loss, metrics=job.metrics)

In [7]:
# Train the Model
model.fit(X_train, y_train, batch_size=job.batchSize, epochs=job.numEpochs, validation_data=(X_test, y_test))

Epoch 1/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 52s 79ms/step - accuracy: 0.8802 - loss: 2.6590 - val_accuracy: 0.9271 - val_loss: 0.1987
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.9085 - loss: 0.2257 - val_accuracy: 0.9401 - val_loss: 0.1572
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.9398 - loss: 0.1587 - val_accuracy: 0.9411 - val_loss: 0.1590
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - accuracy: 0.9559 - loss: 0.1141 - val_accuracy: 0.9758 - val_loss: 0.0733
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - accuracy: 0.9718 - loss: 0.0800 - val_accuracy: 0.9752 - val_loss: 0.0755
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - accuracy: 0.9753 - loss: 0.0694 - val_accuracy: 0.9732 - val_loss: 0.0783
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 50s 79ms/step - accuracy: 0.9768 - loss: 0.0614 - val_accuracy: 0.9785 - val_loss: 0.0654
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 82s 79ms/step - accuracy: 0.9841 - loss: 0.0437 - 

In [8]:
joblib.dump(model, job.persistedModel)

['ASVspoof-2019-1_2025-03-16T21-46-18.789612.libjob']

### Test Model

In [9]:
import joblib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import configuration.configuration as configuration
from notebook_utils import notebookToPython
from readers.label_reader import readLabelsWithJob

In [10]:
oldJob = job
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)
newPersistedModelResults = job.persistedModelResults

if (len(oldJob.persistedModel) > 0 and oldJob.persistedModel != job.persistedModel):
    job.persistedModel = oldJob.persistedModel
    job.persistedModelResults = oldJob.persistedModelResults
    print(f"Loading previously generated model: {job.persistedModel}")


Write python file
Generating new model name: ASVspoof-2019-1_2025-03-16T22-04-26.041701.libjob
Assigned model name: ASVspoof-2019-1_2025-03-16T22-04-26.041701.libjob
Loading previously generated model: ASVspoof-2019-1_2025-03-16T21-46-18.789612.libjob


In [11]:
model = joblib.load(job.persistedModel)

In [12]:
if (not (len(X_test ) > 0 and len(y_test) > 0)):
    generator = MelSpectrogramGenerator()
    X, y_encoded = generator.generateMelSpectrograms(job, job.dataPathSuffix)
    X_test = X
    y_test = y_encoded
    job.persistedModelResults = newPersistedModelResults

In [13]:
y_pred = model.predict(X_test)
y_pred_work = np.argmax(y_pred, axis=1)
y_test_work = np.argmax(y_test, axis=1)
y_pred_work

159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step


array([1, 0, 0, ..., 0, 0, 0])

In [14]:
y_test_work

array([1, 0, 0, ..., 0, 1, 0])

In [15]:
from datetime import datetime
import json
import pytz

score = accuracy_score(y_test_work, y_pred_work)

timestamp_utc = datetime.now(pytz.utc)

In [16]:
prettyJson = json.dumps(job.__dict__, indent=4)

report = f"job completed: {timestamp_utc.isoformat()}\n"
report = report + f"model file: {job.persistedModel}\n"
report = report + f"accuracy_score: {score}\n\n"
report = report + f"job: {prettyJson}\n"

print(report)

with open(job.persistedModelResults, "w") as file:
    file.write(report)

job completed: 2025-03-17T03:04:30.002190+00:00
model file: ASVspoof-2019-1_2025-03-16T21-46-18.789612.libjob
accuracy_score: 0.9816784869976359

job: {
    "jobId": "ASVspoof-2019-1",
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_train/flac",
    "dataExtension": ".flac",
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt",
    "executeToCategoricalForLabels": true,
    "numClasses": 2,
    "sampleRate": 16000,
    "duration": 5,
    "numMels": 128,
    "maxTimeSteps": 109,
    "optimizer": "adam",
    "loss": "categorical_crossentropy",
    "metrics": [
        "accuracy"
    ],
    "batchSize": 32,
    "numEpochs": 10,
    "persistedModel": "ASVspoof-2019-1_2025-03-16T21-46-18.789612.libjob",
    "persistedModelResults": "ASVspoof-2019-1_2025-03-16T21-46-18.789612.txt"
}

